# Doladenie modelov Open AI

Tento notebook je založený na aktuálnych pokynoch uvedených v dokumentácii [Doladenie](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) od Open AI.

Doladenie zlepšuje výkon základných modelov pre vašu aplikáciu tým, že ich znovu trénuje s ďalšími údajmi a kontextom relevantným pre konkrétny prípad použitia alebo scenár. Všimnite si, že techniky návrhu promptov ako _few shot learning_ a _retrieval augmented generation_ vám umožňujú vylepšiť predvolený prompt relevantnými údajmi na zlepšenie kvality. Tieto prístupy sú však obmedzené maximálnou veľkosťou tokenového okna cieľového základného modelu.

S doladením efektívne znovu trénujeme samotný model s požadovanými údajmi (čo nám umožňuje použiť oveľa viac príkladov, než sa zmestí do maximálneho tokenového okna) - a nasadzujeme _vlastnú_ verziu modelu, ktorá už nepotrebuje mať príklady poskytnuté počas inferencie. To nielen zlepšuje efektívnosť nášho návrhu promptu (máme väčšiu flexibilitu pri používaní tokenového okna na iné veci), ale potenciálne tiež zlepšuje naše náklady (znížením počtu tokenov, ktoré musíme poslať modelu počas inferencie).

Doladenie má 4 kroky:
1. Pripraviť tréningové údaje a nahrať ich.
1. Spustiť tréningovú úlohu na získanie doladeného modelu.
1. Vyhodnotiť doladený model a iterovať pre kvalitu.
1. Nasadiť doladený model na inferenciu, keď sme spokojní.

Všimnite si, že nie všetky základné modely podporujú doladenie - [skontrolujte dokumentáciu OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) pre najnovšie informácie. Môžete tiež doladiť predtým doladený model. V tomto návode použijeme `gpt-35-turbo` ako náš cieľový základný model pre doladenie.

---


### Krok 1.1: Pripravte si svoj dataset

Postavme chatbot, ktorý vám pomôže pochopiť periodickú tabuľku prvkov tým, že bude odpovedať na otázky o prvku pomocou limeriku. V _tomto_ jednoduchom návode vytvoríme len dataset na trénovanie modelu s niekoľkými ukážkovými príkladmi odpovedí, ktoré ukazujú očakávaný formát dát. V reálnom použití by ste potrebovali vytvoriť dataset s oveľa väčším počtom príkladov. Môžete tiež použiť otvorený dataset (pre vašu aplikačnú oblasť), ak existuje, a preformátovať ho na použitie pri doladení.

Keďže sa zameriavame na `gpt-35-turbo` a hľadáme odpoveď v jednom kole (chat completion), môžeme vytvoriť príklady pomocou [tohto navrhovaného formátu](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), ktorý odráža požiadavky OpenAI na chat completion. Ak očakávate viackolovú konverzáciu, použili by ste [formát príkladu pre viackolo](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), ktorý obsahuje parameter `weight` na signalizovanie, ktoré správy by mali byť použité (alebo nie) pri doladení.

Pre náš návod použijeme jednoduchší formát jedného kola. Dáta sú vo formáte [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) s 1 záznamom na riadok, každý reprezentovaný ako JSON objekt. Nižšie uvedený úryvok ukazuje 2 záznamy ako ukážku - pozrite si [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) pre celý ukážkový súbor (10 príkladov), ktorý použijeme v našom návode na doladenie. **Poznámka:** Každý záznam _musí_ byť definovaný v jednom riadku (nie rozdelený na viac riadkov ako je bežné v naformátovanom JSON súbore).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

V reálnom použití budete potrebovať oveľa väčší súbor príkladov pre dobré výsledky - kompromis bude medzi kvalitou odpovedí a časom/nákladmi na doladenie. Používame malý súbor, aby sme mohli doladenie rýchlo dokončiť a ilustrovať proces. Pozrite si [tento príklad z OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) pre zložitejší návod na doladenie.


---

### Krok 1.2 Nahrajte svoj dataset

Nahrajte dáta pomocou Files API [ako je popísané tu](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Upozorňujeme, že na spustenie tohto kódu musíte najskôr vykonať nasledujúce kroky:
 - Nainštalovať Python balík `openai` (uistite sa, že používate verziu >=0.28.0 pre najnovšie funkcie)
 - Nastaviť premennú prostredia `OPENAI_API_KEY` na váš OpenAI API kľúč
Ak sa chcete dozvedieť viac, pozrite si [návod na nastavenie](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) poskytnutý pre kurz.

Teraz spustite kód na vytvorenie súboru na nahranie z vášho lokálneho JSONL súboru.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Krok 2.1: Vytvorte úlohu doladenia pomocou SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Krok 2.2: Skontrolujte stav úlohy

Tu je niekoľko vecí, ktoré môžete urobiť s API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Zoznam posledných n úloh doladenia
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Získajte podrobnosti o konkrétnej úlohe doladenia
- `client.fine_tuning.jobs.cancel(<job_id>)` - Zrušte úlohu doladenia
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Zoznam až n udalostí z úlohy
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prvým krokom procesu je _overenie tréningového súboru_, aby sa zabezpečilo, že údaje sú v správnom formáte.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Krok 2.3: Sledovanie udalostí na monitorovanie pokroku


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Krok 2.4: Zobraziť stav v OpenAI Dashboard


Stav si môžete pozrieť aj na webovej stránke OpenAI v sekcii _Fine-tuning_ platformy. Táto sekcia vám ukáže stav aktuálnej úlohy a tiež umožní sledovať históriu predchádzajúcich spustení úloh. Na tomto snímku obrazovky vidíte, že predchádzajúce spustenie zlyhalo a druhé spustenie bolo úspešné. Pre kontext, k tomu došlo, keď prvé spustenie použilo JSON súbor s nesprávne naformátovanými záznamami – po oprave druhé spustenie úspešne dokončilo a model bol sprístupnený na použitie.

![Fine-tuning job status](../../../../../translated_images/sk/fine-tuned-model-status.563271727bf7bfba.png)


Stavové správy a metriky si môžete pozrieť aj posunutím nižšie vo vizuálnom paneli, ako je znázornené:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/sk/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/sk/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### Krok 3.1: Získajte ID a otestujte doladený model v kóde


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Krok 3.2: Načítanie a testovanie doladeného modelu v Playground

Teraz môžete otestovať doladený model dvoma spôsobmi. Najprv môžete navštíviť Playground a použiť rozbaľovaciu ponuku Models na výber vášho novo doladeného modelu zo zoznamu možností. Druhou možnosťou je použiť možnosť "Playground" zobrazenú v paneli Fine-tuning (pozri snímku obrazovky vyššie), ktorá spustí nasledujúce _porovnávacie_ zobrazenie, ktoré zobrazuje základnú a doladenú verziu modelu vedľa seba pre rýchle vyhodnotenie.

![Fine-tuning job status](../../../../../translated_images/sk/fine-tuned-playground-compare.56e06f0ad8922016.png)

Jednoducho vyplňte systémový kontext použitý vo vašich tréningových dátach a zadajte vašu testovaciu otázku. Všimnete si, že obe strany sú aktualizované s identickým kontextom a otázkou. Spustite porovnanie a uvidíte rozdiel vo výstupoch medzi nimi. _Všimnite si, ako doladený model zobrazuje odpoveď vo formáte, ktorý ste poskytli vo svojich príkladoch, zatiaľ čo základný model jednoducho nasleduje systémový prompt_.

![Fine-tuning job status](../../../../../translated_images/sk/fine-tuned-playground-launch.5a26495c983c6350.png)

Všimnete si, že porovnanie tiež poskytuje počet tokenov pre každý model a čas potrebný na inferenciu. **Tento konkrétny príklad je jednoduchý a slúži na ukážku procesu, ale v skutočnosti neodráža reálnu dátovú sadu alebo scenár**. Môžete si všimnúť, že oba príklady ukazujú rovnaký počet tokenov (systémový kontext a používateľský prompt sú identické) a doladený model potrebuje viac času na inferenciu (vlastný model).

V reálnych scenároch nebudete používať takýto jednoduchý príklad, ale doladenie na základe reálnych dát (napr. katalóg produktov pre zákaznícku podporu), kde bude kvalita odpovede oveľa zreteľnejšia. V _tomto_ kontexte dosiahnutie rovnakej kvality odpovede so základným modelom bude vyžadovať viac vlastného prompt inžinierstva, čo zvýši použitie tokenov a potenciálne aj súvisiaci čas spracovania inferencie. _Ak to chcete vyskúšať, pozrite si príklady doladenia v OpenAI Cookbook, aby ste mohli začať._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zrieknutie sa zodpovednosti**:
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, majte na pamäti, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Originálny dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
